In [ ]:
from keras.layers import Convolution2D,MaxPooling2D,Dropout,Dense,Flatten,GlobalAveragePooling2D
from keras import models,losses,optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.models import Sequential
import os

In [ ]:
base_model = applications.resnet50.ResNet50(weights= 'imagenet', include_top=False, input_shape= (48,48,3))
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu', name='Dense_Intermediate'))
model.add(Dropout(0.1, name='Dropout_Regularization'))
model.add(Dense(24, activation='sigmoid', name='Output'))

In [ ]:
for cnn_block in model3.layers[0].layers:
    cnn_block.trainable = False
model3.layers[0].trainable = False

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=20,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,vertical_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory('dataset/train/',target_size=(48, 48),batch_size=10,class_mode="categorical",color_mode='rgb',shuffle= True)
test_set = test_datagen.flow_from_directory('dataset/test',target_size=(48, 48),batch_size=1,class_mode="categorical",color_mode='rgb',shuffle= False)

In [ ]:
model.compile(optimizer='adam',  loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
labels_count = dict()
for img_class in [ic for ic in os.listdir('dataset/train/') if ic[0] != '.']:
    labels_count[img_class] = len(os.listdir('dataset/train/' + img_class))
total_count = sum(labels_count.values())
class_weights = {cls: total_count / count for cls, count in 
                 enumerate(labels_count.values())}


model.fit_generator(
    train_set,
    steps_per_epoch=len(train_set.filenames) // 10,
    epochs=10,
    validation_data=test_set,
    validation_steps=len(test_set.filenames) // 1,
    class_weight=class_weights
)

In [ ]:
json_mod = model.to_json()
#write json file
with open('model_RESNET50.json','w') as file:
    file.write(json_mod)
#Save weights    
model.save_weights('model_RESNET50.h5')

In [ ]:
model.summary()

In [ ]:
base_model.summary()